In [1]:
import pandas as pd
import numpy as np
import h5py
import pyarrow as pa
import pyarrow.parquet as pq

from ulmo import io as ulmo_io

1. Copy a parquet file and rename
2. Open with pandas

In [2]:
tbl = pd.read_parquet('/home/jovyan/ulmo/ulmo/tests/test.parquet')
tbl.head()

,modis_lat,modis_lon,modis_col,modis_datetime,modis_LL,modis_row,clear_fraction,mean_temperature,Tmin,Tmax,...,field_size,pp_type,pp_idx,lat,lon,row,col,datetime,filename,LL
0,-52.410507,-42.126858,1028,2012-01-01 04:10:00,-147.370605,64,0.049805,4.126851,3.401999,6.411,...,64,0,296266,-52.406319,-42.135418,4687,17049,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,127.225403
1,-52.194720,-42.787216,997,2012-01-01 04:10:00,-248.946106,48,0.043213,5.315916,3.464000,6.544,...,64,0,592057,-52.188812,-42.781250,4705,17018,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,53.644531
2,-52.031720,-43.191420,976,2012-01-01 04:10:00,-264.156342,33,0.049500,5.914161,3.557000,6.852,...,64,0,214472,-52.031147,-43.197918,4718,16998,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,161.347656
3,-51.908875,-42.702328,997,2012-01-01 04:10:00,-228.816925,16,0.018188,5.859090,3.442000,6.709,...,64,0,467098,-51.909538,-42.697918,4728,17022,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,172.184753
4,-51.865665,-43.186947,974,2012-01-01 04:10:00,-260.829315,16,0.048279,6.130058,5.386000,6.852,...,64,0,415175,-51.860813,-43.177082,4732,16999,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,156.553528


In [3]:
tbl.iloc[0]

modis_lat                                                  -52.410507
modis_lon                                                  -42.126858
modis_col                                                        1028
modis_datetime                                    2012-01-01 04:10:00
modis_LL                                                  -147.370605
modis_row                                                          64
clear_fraction                                               0.049805
mean_temperature                                             4.126851
Tmin                                                         3.401999
Tmax                                                            6.411
T90                                                              5.46
T10                                                             3.562
modis_filename                   AQUA_MODIS.20120101T041007.L2.SST.nc
UID                                                132537589513787314
pp_file             

3. Drop the columns you need to replace: LL, Tmin, Tmax, T90, T10

In [4]:
tbl1 = tbl.drop( columns = ['LL', 'Tmin', 'Tmax', 'T90', 'T10'])

In [5]:
tbl1.head()

,modis_lat,modis_lon,modis_col,modis_datetime,modis_LL,modis_row,clear_fraction,mean_temperature,modis_filename,UID,...,pp_root,field_size,pp_type,pp_idx,lat,lon,row,col,datetime,filename
0,-52.410507,-42.126858,1028,2012-01-01 04:10:00,-147.370605,64,0.049805,4.126851,AQUA_MODIS.20120101T041007.L2.SST.nc,132537589513787314,...,llc_std,64,0,296266,-52.406319,-42.135418,4687,17049,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...
1,-52.194720,-42.787216,997,2012-01-01 04:10:00,-248.946106,48,0.043213,5.315916,AQUA_MODIS.20120101T041007.L2.SST.nc,132537805313721278,...,llc_std,64,0,592057,-52.188812,-42.781250,4705,17018,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...
2,-52.031720,-43.191420,976,2012-01-01 04:10:00,-264.156342,33,0.049500,5.914161,AQUA_MODIS.20120101T041007.L2.SST.nc,132537968313680858,...,llc_std,64,0,214472,-52.031147,-43.197918,4718,16998,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...
3,-51.908875,-42.702328,997,2012-01-01 04:10:00,-228.816925,16,0.018188,5.859090,AQUA_MODIS.20120101T041007.L2.SST.nc,132538091113729767,...,llc_std,64,0,467098,-51.909538,-42.697918,4728,17022,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...
4,-51.865665,-43.186947,974,2012-01-01 04:10:00,-260.829315,16,0.048279,6.130058,AQUA_MODIS.20120101T041007.L2.SST.nc,132538134313681305,...,llc_std,64,0,415175,-51.860813,-43.177082,4732,16999,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...


4. Push the preproc file into s3
5. Re-write columns: pp_file

In [6]:
tbl1.pp_file = 's3://llc/PreProc/LLC_modis2012_noise_track_preproc.h5'

In [7]:
tbl1.pp_file

0         s3://llc/PreProc/LLC_modis2012_noise_track_pre...
1         s3://llc/PreProc/LLC_modis2012_noise_track_pre...
2         s3://llc/PreProc/LLC_modis2012_noise_track_pre...
3         s3://llc/PreProc/LLC_modis2012_noise_track_pre...
4         s3://llc/PreProc/LLC_modis2012_noise_track_pre...
                                ...                        
635290    s3://llc/PreProc/LLC_modis2012_noise_track_pre...
635291    s3://llc/PreProc/LLC_modis2012_noise_track_pre...
635292    s3://llc/PreProc/LLC_modis2012_noise_track_pre...
635293    s3://llc/PreProc/LLC_modis2012_noise_track_pre...
635294    s3://llc/PreProc/LLC_modis2012_noise_track_pre...
Name: pp_file, Length: 635295, dtype: object

6. Add columns: DT (absolute), DT_p (90th percentile - 10th percentile)

In [8]:
# create the data
DT = np.empty( (635295,))
DT_p = np.empty( (635295,))

with h5py.File('/home/jovyan/LLC_modis2012_noise_track_preproc.h5', 'r') as f:
    
    imgs = f['valid'][:]

print_out_list = np.arange(0, 635295, 50000)

for idx in range(635295):
    img = imgs[idx, 0, ...]
    DT[idx] = np.max(img) - np.min(img)
    DT_p[idx] = np.percentile(img, 90) - np.percentile(img, 10)
    
    if idx in print_out_list:
        print(' At {} out of 635395'.format(idx))

 At 0 out of 635395
 At 50000 out of 635395
 At 100000 out of 635395
 At 150000 out of 635395
 At 200000 out of 635395
 At 250000 out of 635395
 At 300000 out of 635395
 At 350000 out of 635395
 At 400000 out of 635395
 At 450000 out of 635395
 At 500000 out of 635395
 At 550000 out of 635395
 At 600000 out of 635395


In [9]:
tbl2 = tbl1.sort_values('pp_idx')
DT_object = pd.Series(data = DT, index = tbl2.index)
DT_pobject = pd.Series(data = DT_p, index = tbl.index)

In [10]:
tbl1['DT'] = DT_object
tbl1['DT_p'] = DT_pobject

7. Look at it

In [12]:
tbl1.head()

,modis_lat,modis_lon,modis_col,modis_datetime,modis_LL,modis_row,clear_fraction,mean_temperature,modis_filename,UID,...,pp_type,pp_idx,lat,lon,row,col,datetime,filename,DT,DT_p
0,-52.410507,-42.126858,1028,2012-01-01 04:10:00,-147.370605,64,0.049805,4.126851,AQUA_MODIS.20120101T041007.L2.SST.nc,132537589513787314,...,0,296266,-52.406319,-42.135418,4687,17049,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.070718,1.955382
1,-52.194720,-42.787216,997,2012-01-01 04:10:00,-248.946106,48,0.043213,5.315916,AQUA_MODIS.20120101T041007.L2.SST.nc,132537805313721278,...,0,592057,-52.188812,-42.781250,4705,17018,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.179813,0.525892
2,-52.031720,-43.191420,976,2012-01-01 04:10:00,-264.156342,33,0.049500,5.914161,AQUA_MODIS.20120101T041007.L2.SST.nc,132537968313680858,...,0,214472,-52.031147,-43.197918,4718,16998,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.374310,0.601560
3,-51.908875,-42.702328,997,2012-01-01 04:10:00,-228.816925,16,0.018188,5.859090,AQUA_MODIS.20120101T041007.L2.SST.nc,132538091113729767,...,0,467098,-51.909538,-42.697918,4728,17022,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.359148,2.826178
4,-51.865665,-43.186947,974,2012-01-01 04:10:00,-260.829315,16,0.048279,6.130058,AQUA_MODIS.20120101T041007.L2.SST.nc,132538134313681305,...,0,415175,-51.860813,-43.177082,4732,16999,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,1.567054,0.337593


8. Upload the table

In [13]:
ulmo_io.write_main_table(tbl1, 's3://llc/Tables/LLC_modis_noise_track.parquet')

Wrote Analysis Table: s3://llc/Tables/LLC_modis_noise_track.parquet


9. After having run parquet table thru ulmo, open file.

In [2]:
tbl_new = ulmo_io.load_main_table('s3://llc/Tables/LLC_modis_noise_track.parquet')
tbl_new.head()

Read main table: s3://llc/Tables/LLC_modis_noise_track.parquet


,modis_lat,modis_lon,modis_col,modis_datetime,modis_LL,modis_row,clear_fraction,mean_temperature,modis_filename,UID,...,pp_type,pp_idx,lat,lon,row,col,datetime,filename,DT,DT_p
0,-52.410507,-42.126858,1028,2012-01-01 04:10:00,-147.370605,64,0.049805,4.126851,AQUA_MODIS.20120101T041007.L2.SST.nc,132537589513787314,...,0,296266,-52.406319,-42.135418,4687,17049,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.070718,1.955382
1,-52.194720,-42.787216,997,2012-01-01 04:10:00,-248.946106,48,0.043213,5.315916,AQUA_MODIS.20120101T041007.L2.SST.nc,132537805313721278,...,0,592057,-52.188812,-42.781250,4705,17018,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.179813,0.525892
2,-52.031720,-43.191420,976,2012-01-01 04:10:00,-264.156342,33,0.049500,5.914161,AQUA_MODIS.20120101T041007.L2.SST.nc,132537968313680858,...,0,214472,-52.031147,-43.197918,4718,16998,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.374310,0.601560
3,-51.908875,-42.702328,997,2012-01-01 04:10:00,-228.816925,16,0.018188,5.859090,AQUA_MODIS.20120101T041007.L2.SST.nc,132538091113729767,...,0,467098,-51.909538,-42.697918,4728,17022,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.359148,2.826178
4,-51.865665,-43.186947,974,2012-01-01 04:10:00,-260.829315,16,0.048279,6.130058,AQUA_MODIS.20120101T041007.L2.SST.nc,132538134313681305,...,0,415175,-51.860813,-43.177082,4732,16999,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,1.567054,0.337593


10. No LL column: add it.

In [3]:
with ulmo_io.open( 's3://llc/Evaluations/LLC_modis2012_noise_track_log_prob.h5', 'rb') as f: 
    pp_hf = h5py.File(f, 'r')
    data = pp_hf['valid'][()]
    pp_hf.close()

In [4]:
order = tbl_new.sort_values('pp_idx').index
order

Int64Index([248464,  41335, 499377, 594819, 414806, 457889, 523802,  77215,
            524092, 295330,
            ...
            428472, 257004, 258642, 162989, 211810, 414618, 299184, 149139,
            595987, 194382],
           dtype='int64', length=635295)

In [5]:
LL_values = pd.Series( data=data, index=order)
LL_values

248464     27.274902
41335     383.768494
499377    256.174561
594819   -283.288177
414806    862.625671
             ...    
414618    571.445129
299184    -32.440491
149139    315.458923
595987    129.725952
194382    121.148743
Length: 635295, dtype: float32

In [6]:
tbl_new['LL'] = LL_values

In [7]:
tbl_new.head()

,modis_lat,modis_lon,modis_col,modis_datetime,modis_LL,modis_row,clear_fraction,mean_temperature,modis_filename,UID,...,pp_idx,lat,lon,row,col,datetime,filename,DT,DT_p,LL
0,-52.410507,-42.126858,1028,2012-01-01 04:10:00,-147.370605,64,0.049805,4.126851,AQUA_MODIS.20120101T041007.L2.SST.nc,132537589513787314,...,296266,-52.406319,-42.135418,4687,17049,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.070718,1.955382,125.505615
1,-52.194720,-42.787216,997,2012-01-01 04:10:00,-248.946106,48,0.043213,5.315916,AQUA_MODIS.20120101T041007.L2.SST.nc,132537805313721278,...,592057,-52.188812,-42.781250,4705,17018,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.179813,0.525892,49.920471
2,-52.031720,-43.191420,976,2012-01-01 04:10:00,-264.156342,33,0.049500,5.914161,AQUA_MODIS.20120101T041007.L2.SST.nc,132537968313680858,...,214472,-52.031147,-43.197918,4718,16998,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.374310,0.601560,160.204773
3,-51.908875,-42.702328,997,2012-01-01 04:10:00,-228.816925,16,0.018188,5.859090,AQUA_MODIS.20120101T041007.L2.SST.nc,132538091113729767,...,467098,-51.909538,-42.697918,4728,17022,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,3.359148,2.826178,166.913330
4,-51.865665,-43.186947,974,2012-01-01 04:10:00,-260.829315,16,0.048279,6.130058,AQUA_MODIS.20120101T041007.L2.SST.nc,132538134313681305,...,415175,-51.860813,-43.177082,4732,16999,2012-01-01,s3://llc/ThetaUVSalt/LLC4320_2012-01-01T00_00_...,1.567054,0.337593,160.269714


11. Rewrite the main table

In [8]:
ulmo_io.write_main_table(tbl_new, 's3://llc/Tables/LLC_modis_noise_track.parquet')

Wrote Analysis Table: s3://llc/Tables/LLC_modis_noise_track.parquet
